# Creating and Valuing a CDS Index Option

In [1]:
# This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
# without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  
# See the license for more details.

This notebook does the valuation of an option on a CDS index using Anderson's method and an Adjusted Black Method. For details see Modelling Singlename and Multiname Credit Derivatives by D.O'Kane.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinMath import ONE_MILLION
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.products.credit.FinCDS import FinCDS
from financepy.market.curves.FinCDSCurve import FinCDSCurve
from financepy.products.credit.FinCDSIndexOption import FinCDSIndexOption
from financepy.products.credit.FinCDSIndexPortfolio import FinCDSIndexPortfolio

In [4]:
tradeDate = FinDate(1, 8, 2007)
stepInDate = tradeDate.addDays(1)
valuationDate = stepInDate
settlementDate = stepInDate

## Build Libor Curve

In [5]:
depos = []
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"1Y",0.0502,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",0.0502,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",0.0501,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",0.0502,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

In [6]:
cdsMaturity3Y = tradeDate.nextCDSDate(36)
cdsMaturity5Y = tradeDate.nextCDSDate(60)
cdsMaturity7Y = tradeDate.nextCDSDate(84)
cdsMaturity10Y = tradeDate.nextCDSDate(120)

## Load the Underlying CDS Index Portfolio

In [7]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
issuerCurves = []

numCredits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    creditName = splitRow[0]
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recoveryRate = float(splitRow[5])
    cds3Y = FinCDS(stepInDate, cdsMaturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, cdsMaturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, cdsMaturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, cdsMaturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    issuerCurves.append(issuerCurve)

## Term Structure of CDS Index Market

We now set up the term structure of CDS Index trades and start by setting up the maturity dates of the index

In [8]:
indexMaturity3Y = FinDate(20,12,2009)
indexMaturity5Y = FinDate(20,12,2011)
indexMaturity7Y = FinDate(20,12,2013)
indexMaturity10Y = FinDate(20,12,2016)

Now we set their upfronts and coupons

In [9]:
indexCoupons = [0.002, 0.0037, 0.0050, 0.0063]
indexUpfronts = [0.0, 0.0, 0.0, 0.0]
indexMaturityDates = [indexMaturity3Y, indexMaturity5Y, indexMaturity7Y, indexMaturity10Y]
indexRecovery = 0.40

## CDS Index Adjustment

Ensure that the underlying CDS are adjusted to agree with index spreads

In [11]:
 adjustedIssuerCurves = FinCDSIndexPortfolio.hazardRateAdjustIntrinsic(valuationDate, issuerCurves, 
                                                                       indexCoupons, indexUpfronts,
                                                                       indexMaturityDates, indexRecovery, 1e-6)

## Setting up the Index Option

In [12]:
expiryDate = FinDate(1,2,2008)
maturityDate = indexMaturity5Y
notional = ONE_MILLION
volatility = 0.50
strike = 0.005

In [19]:
indexCoupon = 0.005

In [20]:
option = FinCDSIndexOption(expiryDate, maturityDate, indexCoupon, strike, notional)

## Anderson Method

This is the full method that takes in the entire vector of adjusted issuer curves

In [21]:
%time v_pay_1, v_rec_1, strikeValue, mu, expH \
        = option.valueAnderson(valuationDate, adjustedIssuerCurves, indexRecovery, volatility)

Wall time: 19.9 ms


In [22]:
print("Payer Value: %8.2f"% v_pay_1)

Payer Value:  1139.62


In [23]:
print("Receiver Value: %8.2f"% v_rec_1)

Receiver Value:  3780.83


## Adjusted Black Method

This is a faster method that gives a similar result

In [24]:
v_pay_2, v_rec_2 = option.valueAdjustedBlack(valuationDate, indexCurve, indexRecovery, liborCurve, volatility)

NameError: name 'indexCurve' is not defined

In [184]:
print("Payer Value: %8.2f"% v_pay_2)

Payer Value:  7036.58


In [185]:
print("Receiver Value: %8.2f"% v_rec_2)

Receiver Value:   706.69


## Comparison of Anderson versus Adjusted Black Model

In [186]:
print("Strike   Coupon    V_PAY_AND    V_PAY_BLK    DIFF(%)     V_REC_AND   V_REC_BLK     DIFF(%)")

for index in np.linspace(20, 60, 5):
    
    # Create a flat CDS index curve
    cdsContracts = []
    for dt in indexMaturityDates:
        cds = FinCDS(valuationDate, dt, index / 10000.0)
        cdsContracts.append(cds)

    # Build the flat CDS index curve
    indexCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, indexRecovery)

    # Now we need to set up the underlying curves
    if 1 == 1:
        
        indexSpreads = [index / 10000.0] * 4
        adjustedIssuerCurves = FinCDSIndexPortfolio.hazardRateAdjustIntrinsic(valuationDate, issuerCurves, 
                                                                              indexSpreads, indexUpfronts,
                                                                              indexMaturityDates, indexRecovery, 
                                                                              1e-5)
    for strike in np.linspace(20,60,5):    

        option = FinCDSIndexOption(expiryDate, maturityDate, indexCoupon, strike / 10000.0, notional)
        v_pay_1, v_rec_1, strikeValue, mu, expH = option.valueAnderson(valuationDate, adjustedIssuerCurves, indexRecovery, volatility)
        v_pay_2, v_rec_2 = option.valueAdjustedBlack(valuationDate, indexCurve, indexRecovery, liborCurve, volatility)
            
        diff1 = 100.0 * (v_pay_1 - v_pay_2)/v_pay_1
        diff2 = 100.0 * (v_rec_1 - v_rec_2)/v_rec_1
        
        print("%6.1f    %6.1f    %8.2f    %8.2f   %8.2f       %8.2f    %8.2f    %8.2f" % 
              (strike, index, v_pay_1, v_pay_2, diff1, v_rec_1, v_rec_2, diff2 ))

Strike   Coupon    V_PAY_AND    V_PAY_BLK    DIFF(%)     V_REC_AND   V_REC_BLK     DIFF(%)
  20.0      20.0     1610.14     1612.67      -0.16         619.03      609.98        1.46
  30.0      20.0      402.96      402.48       0.12        2879.47     2864.51        0.52
  40.0      20.0       87.74       89.08      -1.52        6010.00     5988.11        0.36
  50.0      20.0       18.48       19.69      -6.53        9364.75     9328.23        0.39
  60.0      20.0        4.23        4.55      -7.47       12752.91    12695.31        0.45
  20.0      30.0     5038.79     5045.39      -0.13          87.82       82.80        5.71
  30.0      30.0     2404.91     2408.26      -0.14         918.65      913.31        0.58
  40.0      30.0      973.29      981.04      -0.80        2929.86     2925.99        0.13
  50.0      30.0      363.76      370.08      -1.74        5741.43     5727.42        0.24
  60.0      30.0      131.34      136.14      -3.66        8908.49     8878.55        0.34